In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
default = pd.read_csv('data\credit_card_default.csv', index_col ='ID')

In [6]:
default.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [12]:
default.columns = default.columns.str.lower()
pay_features = ['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']
for pay in pay_features:
    default.loc[default[pay]<=0, pay] = 0
default.rename(columns={'default payment next month' : 'default'}, inplace =True)
default.head()

,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,pay_5,...,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,0,0,0,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,0,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,0,0,0,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [15]:
default['grad_school'] =(default['education']==1).astype('int')
default['university'] =(default['education']==1).astype('int')
default['high_school'] =(default['education']==1).astype('int')
default['male'] =(default['sex']==1).astype('int')
default['married'] =(default['marriage']==1).astype('int')
default.drop(['education', 'sex', 'marriage'] , axis=1, inplace=True)
default.head()



,limit_bal,age,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,...,pay_amt3,pay_amt4,pay_amt5,pay_amt6,default,grad_school,university,high_school,male,married
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,24,2,2,0,0,0,0,3913,3102,...,0,0,0,0,1,0,0,0,0,1
2,120000,26,0,2,0,0,0,2,2682,1725,...,1000,1000,0,2000,1,0,0,0,0,0
3,90000,34,0,0,0,0,0,0,29239,14027,...,1000,1000,1000,5000,0,0,0,0,0,0
4,50000,37,0,0,0,0,0,0,46990,48233,...,1200,1100,1069,1000,0,0,0,0,0,1
5,50000,57,0,0,0,0,0,0,8617,5670,...,10000,9000,689,679,0,0,0,0,1,1


In [21]:
from sklearn.linear_model import LogisticRegression
x = default[['pay_amt6', 'age']]
y = default['default']
logreg = LogisticRegression()
logreg.fit(x, y)
print(logreg.intercept_)
print(logreg.coef_)

[-1.33997627]
[[-1.41233276e-05  3.97680969e-03]]


In [23]:
def Z(pay_amt, age):
    return logreg.intercept_[0] + logreg.coef_[0][0] * pay_amt + logreg.coef_[0][1] * age

def prob_default(pay_amt, age):
    z= Z(pay_amt, age)
    return 1/(1 + np.exp(-z))

prob_default(pay_amt=0, age=40)

0.23488971532182748

In [27]:
from sklearn.model_selection import train_test_split
target_name = 'default'
x = default.drop('default', axis=1)
y = default[target_name]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [28]:
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(max_iter=1000)
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)
accuracy = accuracy_score(y_pred=y_pred, y_true=y_test)
accuracy

0.7831666666666667